<a href="https://colab.research.google.com/github/ClearSpear/cameramouse/blob/master/handdetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount Google Drive to save files

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/My Drive/Stanford/EE292D/project/colab'

/content/gdrive/My Drive/Stanford/EE292D/project/colab


## Check remaining time on colab session:

In [ ]:
import time, psutil
Start = time.time()- psutil.boot_time()
Left= 12*3600 - Start
print('Time remaining for this session is: ', Left/3600)

Time remaining for this session is:  11.947670471138425


## Download TF object detection API

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print(tf.__version__)

Found GPU at: /device:GPU:0
1.15.2


In [ ]:
tf.enable_eager_execution()

In [ ]:
!git clone https://github.com/tensorflow/models.git

fatal: destination path 'models' already exists and is not an empty directory.


In [ ]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
!pip install tf_slim

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-tk is already the newest version (2.7.17-1~18.04).
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 32 not upgraded.
Need to get 1,818 kB of archives.
After this operation, 7,685 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python-bs4 all 4.6.0-1 [67.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd

In [ ]:
%cd /content/gdrive/My Drive/Stanford/EE292D/project/colab/models/research/
!protoc object_detection/protos/*.proto --python_out=.

/content/gdrive/My Drive/Stanford/EE292D/project/colab/models/research
object_detection/protos/input_reader.proto: warning: Import object_detection/protos/image_resizer.proto but not used.


In [ ]:
import os
os.environ['PYTHONPATH'] += ':/content/gdrive/My Drive/Stanford/EE292D/project/colab/models/research/:/content/gdrive/My Drive/Stanford/EE292D/project/colab/models/research/slim'

In [ ]:
!python setup.py build
!python setup.py install

running build
running build_py
copying object_detection/protos/anchor_generator_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/bipartite_matcher_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/argmax_matcher_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/eval_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/box_coder_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/box_predictor_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/calibration_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/faster_rcnn_box_coder_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/flexible_grid_anchor_generator_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/faster_rcnn_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/graph_rewriter_pb2.py

Test installation was successful:

In [ ]:
%cd /content/gdrive/My Drive/Stanford/EE292D/project/colab/models/research/object_detection/builders/
!python model_builder_test.py

/content/gdrive/My Drive/Stanford/EE292D/project/colab/models/research/object_detection/builders


## Setup to train hand detection model

In [ ]:
%cd /content/gdrive/My Drive/Stanford/EE292D/project/colab

/content/gdrive/My Drive/Stanford/EE292D/project/colab


In [ ]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_11_06_2017.tar.gz
!tar -xvf ssd_mobilenet_v1_coco_11_06_2017.tar.gz

--2020-05-27 21:52:12--  http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_11_06_2017.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 108.177.126.128, 2a00:1450:4013:c00::80
Connecting to download.tensorflow.org (download.tensorflow.org)|108.177.126.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 128048406 (122M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v1_coco_11_06_2017.tar.gz’

ssd_mobilenet_v1_co 100%[===================>] 122.12M  70.0MB/s    in 1.7s    

2020-05-27 21:52:13 (70.0 MB/s) - ‘ssd_mobilenet_v1_coco_11_06_2017.tar.gz’ saved [128048406/128048406]

ssd_mobilenet_v1_coco_11_06_2017/
ssd_mobilenet_v1_coco_11_06_2017/model.ckpt.index
ssd_mobilenet_v1_coco_11_06_2017/model.ckpt.meta
ssd_mobilenet_v1_coco_11_06_2017/frozen_inference_graph.pb
ssd_mobilenet_v1_coco_11_06_2017/model.ckpt.data-00000-of-00001
ssd_mobilenet_v1_coco_11_06_2017/graph.pbtxt


In [ ]:
%load_ext tensorboard
%tensorboard --logdir training/

## Train

In [ ]:
!python models/research/object_detection/train.py --logtostderr --train_dir=training/ --pipeline_config_path=training/ssd_mobilenet_v1_coco.config

Streaming output truncated to the last 5000 lines.
I0602 21:14:49.970083 140248982009600 supervisor.py:1117] Saving checkpoint to path training/model.ckpt
INFO:tensorflow:global step 4270: loss = 3.3401 (2.265 sec/step)
I0602 21:14:52.211557 140252273850240 learning.py:512] global step 4270: loss = 3.3401 (2.265 sec/step)
INFO:tensorflow:Recording summary at step 4271.
I0602 21:14:54.843316 140248998795008 supervisor.py:1050] Recording summary at step 4271.
INFO:tensorflow:global step 4271: loss = 3.4552 (2.627 sec/step)
I0602 21:14:54.847090 140252273850240 learning.py:512] global step 4271: loss = 3.4552 (2.627 sec/step)
INFO:tensorflow:global step 4272: loss = 3.0183 (2.004 sec/step)
I0602 21:14:56.858472 140252273850240 learning.py:512] global step 4272: loss = 3.0183 (2.004 sec/step)
INFO:tensorflow:global step 4273: loss = 3.5261 (2.022 sec/step)
I0602 21:14:58.884292 140252273850240 learning.py:512] global step 4273: loss = 3.5261 (2.022 sec/step)
INFO:tensorflow:global step 427

## Test that the frozen graph works

Create frozen graph for testing locally

In [ ]:
!python models/research/object_detection/export_inference_graph.py \
--input_type image_tensor \
--pipeline_config_path training/ssd_mobilenet_v1_coco.config \
--trained_checkpoint_prefix training/model.ckpt-18937 \
--output_directory trained_inference_graph/

Instructions for updating:
Please use `layer.__call__` method instead.
W0602 06:58:22.666155 140533628540800 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0602 06:58:23.979808 140533628540800 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0602 06:58:24.013973 140533628540800 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0602 06:58:24.046705 140533628540800 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

In [ ]:
!cd /content/gdrive/My\ Drive/Stanford/EE292D/project/colab/models/research

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
#sys.path.append(\"../\")
from object_detection.utils import ops as utils_ops
#from object_detection.utils import label_map_util
#from object_detection.utils import visualization_utils as vis_util

### Model preparation variable
MODEL_NAME = 'trained_inference_graph'
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = 'training/handdetection_label_map.pbtxt'
NUM_CLASSES = 1 #remember number of objects you are training? cool.


### Load a (frozen) Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

###Loading label map
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

### Load image into numpy function
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

###STATING THE PATH TO IMAGES TO BE TESTED
PATH_TO_TEST_IMAGES_DIR = 'test_images/'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 4) ]
IMAGE_SIZE = (12, 8)

## Function to run inference on a single image which will later be used in an iteration
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: image})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.int64)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

### To iterate on each image in the test image path defined 
### NB define the range of numbers and let it match the number of imAGES IN TEST FOLDER +1
for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=1)
  display(Image.fromarray(image_np))

ModuleNotFoundError: ignored

## Convert saved model to frozen graph compatible with tflite

In [ ]:
H = 480
W = 640

In [ ]:
!python models/research/object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path=training/ssd_mobilenet_v1_coco.config \
--trained_checkpoint_prefix=training/model.ckpt-18937 \
--output_directory=tflite \
--add_postprocessing_op=true

Instructions for updating:
Please use `layer.__call__` method instead.
W0601 23:30:43.574209 140496612235136 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0601 23:30:45.029119 140496612235136 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0601 23:30:45.060245 140496612235136 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0601 23:30:45.090565 140496612235136 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

## Convert the tflite model, quantizing using the representative dataset.

In [ ]:
gf = tf.GraphDef()   
m_file = open('tflite/tflite_graph.pb','rb')
gf.ParseFromString(m_file.read())

for n in gf.node:
    print( n.name )
    tensor = n.op # save the input and output tensor

normalized_input_image_tensor
FeatureExtractor/MobilenetV1/Conv2d_0/weights
FeatureExtractor/MobilenetV1/Conv2d_0/weights/read
FeatureExtractor/MobilenetV1/MobilenetV1/Conv2d_0/Conv2D
FeatureExtractor/MobilenetV1/Conv2d_0/BatchNorm/gamma
FeatureExtractor/MobilenetV1/Conv2d_0/BatchNorm/gamma/read
FeatureExtractor/MobilenetV1/Conv2d_0/BatchNorm/beta
FeatureExtractor/MobilenetV1/Conv2d_0/BatchNorm/beta/read
FeatureExtractor/MobilenetV1/Conv2d_0/BatchNorm/moving_mean
FeatureExtractor/MobilenetV1/Conv2d_0/BatchNorm/moving_mean/read
FeatureExtractor/MobilenetV1/Conv2d_0/BatchNorm/moving_variance
FeatureExtractor/MobilenetV1/Conv2d_0/BatchNorm/moving_variance/read
FeatureExtractor/MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/FusedBatchNormV3
FeatureExtractor/MobilenetV1/MobilenetV1/Conv2d_0/Relu6
FeatureExtractor/MobilenetV1/Conv2d_1_depthwise/depthwise_weights
FeatureExtractor/MobilenetV1/Conv2d_1_depthwise/depthwise_weights/read
FeatureExtractor/MobilenetV1/MobilenetV1/Conv2d_1_depthwise/dept

In [ ]:
# A generator that provides a representative dataset
def representative_data_gen():
  dataset_list = tf.data.Dataset.list_files('egohands/train/*')
  for i in range(100):
    image = next(iter(dataset_list))
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [H, W])
    image = tf.cast(image / 255., tf.float32)
    image = tf.expand_dims(image, 0)
    yield [image]

input_tensors = ['normalized_input_image_tensor']
output_tensors = ['TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3']

converter = tf.lite.TFLiteConverter.from_frozen_graph('tflite/tflite_graph.pb', input_tensors, output_tensors, input_shapes={'normalized_input_image_tensor':[1,H,W,3]})
converter.allow_custom_ops = True
converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
#converter.inference_output_type = tf.uint8
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()

with open("converted_model.tflite", "wb") as f:
  f.write(tflite_model)